### RefAV Tutorial
To start, we separate the scenario mining ground truth annotations into separate log folders.

At the same time, we create the ground truth .pkl files we can use to evaluate performance later. 

In [ ]:
from pathlib import Path
import os
from refAV.paths import AV2_DATA_DIR, SM_DATA_DIR
from refAV.dataset_conversion import separate_scenario_mining_annotations, create_gt_mining_pkls_parallel

sm_val_feather = Path('av2_sm_downloads/scenario_mining_val_annotations.feather')
separate_scenario_mining_annotations(sm_val_feather, SM_DATA_DIR / 'val')
create_gt_mining_pkls_parallel(sm_val_feather, SM_DATA_DIR / 'val', num_processes=max(1, int(.9*os.cpu_count())))

Reading input feather file: av2_sm_downloads/scenario_mining_val_annotations.feather
Found 149 unique log IDs
Saved output/sm_dataset/val/ff0dbfc5-8a7b-3a6e-8936-e5e812e45408/sm_annotations.feather
Saved output/sm_dataset/val/05fa5048-f355-3274-b565-c0ddc547b315/sm_annotations.feather
Saved output/sm_dataset/val/51bbdd4d-3065-34ae-b369-b6e0444f34db/sm_annotations.feather
Saved output/sm_dataset/val/b2053fdc-0b94-30bc-aee7-5bc6fb7e9f52/sm_annotations.feather
Saved output/sm_dataset/val/22052525-4f85-3fe8-9d7d-000a9fffce36/sm_annotations.feather
Saved output/sm_dataset/val/7039e410-b5ab-35aa-96bc-2c4b89d3c5e3/sm_annotations.feather
Saved output/sm_dataset/val/36aec72e-5086-376c-b109-295b128e77e1/sm_annotations.feather
Saved output/sm_dataset/val/b213af37-7d89-342d-ae39-8a3c72159a01/sm_annotations.feather
Saved output/sm_dataset/val/0fb7276f-ecb5-3e5b-87a8-cc74c709c715/sm_annotations.feather
Saved output/sm_dataset/val/dfc32963-1524-34f4-9f5e-0292f0f223ae/sm_annotations.feather
Saved outp

Processing log-prompt pairs: 100%|██████████| 1467/1467 [00:00<00:00, 2971049.72it/s]


Completed processing 1467 log-prompt combinations


['Scenario pkl file for parked vehicle_ff0dbfc5 saved successfully.',
 'Scenario pkl file for multiple pedestrians crossing ahead_ff0dbfc5 saved successfully.',
 'Scenario pkl file for pedestrian facing toward ego vehicle_ff0dbfc5 saved successfully.',
 'Scenario pkl file for ego vehicle approaching crossing pedestrians_ff0dbfc5 saved successfully.',
 'Scenario pkl file for braking vehicle_ff0dbfc5 saved successfully.',
 'Scenario pkl file for any vehicle in front of stop sign_ff0dbfc5 saved successfully.',
 "Scenario pkl file for object that hasn't moved_ff0dbfc5 saved successfully.",
 'Scenario pkl file for pedestrian near bollard_ff0dbfc5 saved successfully.',
 'Scenario pkl file for moving vehicles that are not on the road_ff0dbfc5 saved successfully.',
 'Scenario pkl file for vehicle two vehicles ahead is braking_05fa5048 saved successfully.',
 'Scenario pkl file for vehicle accelerating more than two meters per second_05fa5048 saved successfully.',
 'Scenario pkl file for vehicle

RefAV takes any base set of predicted tracks runs a set of filtering operations to identify the relevant portions of each track.
The baseline uses the track predictions from the winner of the 2024  AV2 End-to-End Forecasting challenge: Le3DE2E. This block downloads the track predictions for the val set from Hugging Face.

In [1]:
from huggingface_hub import hf_hub_download
from refAV.dataset_conversion import pickle_to_feather
from refAV.paths import SM_PRED_DIR, AV2_DATA_DIR
from pathlib import Path

repo_id = "CainanD/AV2_Tracker_Predictions"
filename = "Le3DE2E_tracking_predictions_val.pkl"
tracker_predictions_dir = 'tracker_predictions'

hf_hub_download(repo_id, filename, repo_type='dataset', local_dir=tracker_predictions_dir)

tracking_val_predictions = Path(tracker_predictions_dir + '/' + filename)
sm_val_feather = Path('av2_sm_downloads/scenario_mining_val_annotations.feather')

pickle_to_feather(AV2_DATA_DIR / 'val', tracking_val_predictions, SM_PRED_DIR / 'val')

/home/crdavids/miniforge3/envs/refav2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading pickle file: tracker_predictions/Le3DE2E_tracking_predictions_val.pkl
filtered from 6576 to 1423 ids
Created feather file: output/sm_predictions/val/96dd6923-994c-3afe-9830-b15bdfd60f64/annotations.feather
Successfully added ego to annotations for log 96dd6923-994c-3afe-9830-b15bdfd60f64.
filtered from 7214 to 1590 ids
Created feather file: output/sm_predictions/val/11ba4e81-c26f-3cd1-827d-b6913bcef64e/annotations.feather
Successfully added ego to annotations for log 11ba4e81-c26f-3cd1-827d-b6913bcef64e.
filtered from 11007 to 1041 ids
Created feather file: output/sm_predictions/val/02a00399-3857-444e-8db3-a8f58489c394/annotations.feather
Successfully added ego to annotations for log 02a00399-3857-444e-8db3-a8f58489c394.
filtered from 6571 to 1286 ids
Created feather file: output/sm_predictions/val/25e5c600-36fe-3245-9cc0-40ef91620c22/annotations.feather
Successfully added ego to annotations for log 25e5c600-36fe-3245-9cc0-40ef91620c22.
filtered from 7843 to 1105 ids
Created fe

RefAV works by constructing compositional functions that can be used to define a scenario.

Here is an example of using the compositional functions to define a scenario corresponding 
to a "moving vehicle behind another vehicle being crossed by a jaywalking pedestrian'. 

In [1]:
from refAV.utils import *
from refAV.paths import SM_PRED_DIR, SM_DATA_DIR
from IPython.display import Video
import pyvista as pv

dataset_dir = SM_DATA_DIR / 'val'
output_dir = SM_PRED_DIR / 'val'
log_id = '0b86f508-5df9-4a46-bc59-5b9536dbde9f'
log_dir = dataset_dir / log_id

description = 'vehicle behind another vehicle being crossed by a jaywalking pedestrian'

peds = get_objects_of_category(log_dir, category='PEDESTRIAN')
peds_on_road = on_road(peds, log_dir)
jaywalking_peds = scenario_not(at_pedestrian_crossing)(peds_on_road, log_dir)

vehicles = get_objects_of_category(log_dir, category='VEHICLE')
moving_vehicles = scenario_and([in_drivable_area(vehicles, log_dir), scenario_not(stationary)(vehicles, log_dir)])
crossed_vehicles = being_crossed_by(moving_vehicles, jaywalking_peds, log_dir)
behind_crossed_vehicle = get_objects_in_relative_direction(crossed_vehicles, moving_vehicles, log_dir,
											direction='backward', max_number=1, within_distance=25)

print(pv.Report())

#Output scenario outputs a .pkl and .mp4 for the predicted tracks during that scenario
output_scenario(behind_crossed_vehicle, description, log_dir, output_dir, visualize=True)

Video('output/sm_predictions/val/0b86f508-5df9-4a46-bc59-5b9536dbde9f/scenario visualizations/moving vehicle behind another vehicle being crossed by a jaywalking pedestrian_n6.mp4')


--------------------------------------------------------------------------------
  Date: Fri Mar 21 06:13:25 2025 EDT

                OS : Linux (Springdale Open Enterprise Linux 8.6)
            CPU(s) : 96
           Machine : x86_64
      Architecture : 64bit
               RAM : 472.2 GiB
       Environment : Jupyter
       File system : ext4
       GPU Details : error
  MathText Support : True

  Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10)
  [GCC 13.3.0]

           pyvista : 0.44.2
               vtk : 9.3.1
             numpy : 2.1.3
        matplotlib : 3.10.1
            scooby : 0.10.0
             pooch : 1.8.2
            pillow : 11.1.0
           imageio : 2.37.0
           IPython : 8.34.0
             scipy : 1.15.2
              tqdm : 4.67.1
      nest_asyncio : 1.6.0
--------------------------------------------------------------------------------
Scenario pkl file for vehicle behind another vehicle being crossed by a jaywalking pedestri

ERROR:root:Could not find a decent config
2025-03-21 06:13:47.390 (  39.333s) [    7F4FEB6B3200]vtkXOpenGLRenderWindow.:256    ERR| vtkXOpenGLRenderWindow (0x55dcf6543d60): Could not find a decent config



: 

Now we know how to define a scenario, let's let an LLM do it for us.

Using this function requires an [Anthropic API](https://www.anthropic.com/api) key!

Since this can get quite expensive,
we provide the predicted scenario definitions in the output/llm_scenario_predictions folder.

In [ ]:
from refAV.scenario_prediction import predict_scenario_from_description
from refAV.paths import LLM_DEF_DIR

prompt = 'vehicle heading toward ego from the side while at an intersection'
predict_scenario_from_description(prompt, LLM_DEF_DIR)

with open(LLM_DEF_DIR/(prompt + '.txt'), 'r') as def_file:
    definition_text = def_file.read()
    print(definition_text)

vehicle heading toward ego from the side while at an intersection definition saved to output/llm_scenario_predictions
description = 'vehicle heading toward ego from the side while at an intersection'

# Get all vehicles and ego vehicle
vehicles = get_objects_of_category(log_dir, category="VEHICLE")
ego = get_objects_of_category(log_dir, category="EGO_VEHICLE")

# Get vehicles that are moving (not parked/stopped)
moving_vehicles = scenario_not(stationary)(vehicles, log_dir)

# Find vehicles that are near an intersection
vehicles_near_intersection = near_intersection(moving_vehicles, log_dir)

# Find vehicles that are approaching from the sides of ego
# We'll check both left and right sides
vehicles_from_left = get_objects_in_relative_direction(ego, vehicles_near_intersection, log_dir, 
                                                     direction='left', 
                                                     within_distance=20)
vehicles_from_right = get_objects_in_relative_direction(ego

With the basics out of the way, let's run evaluation on the entire validation dataset.
The create_base_prediction function calls the LLM scenario definition generator and the
 runs the defintion to find instance of the prompt.
  It can take quite a bit of time to go through all of the logs.

In [1]:
import json
from tqdm import tqdm
from refAV.eval import create_baseline_prediction

log_prompt_input_path = Path('av2_sm_downloads/log_prompt_pairs_val.json')
with open(log_prompt_input_path, 'rb') as f:
	log_prompts = json.load(f)

for log_id, prompts in tqdm(log_prompts.items()):
	for prompt in prompts:
		create_baseline_prediction(prompt, log_id, SM_PRED_DIR / 'val', LLM_DEF_DIR)

ModuleNotFoundError: No module named 'trackeval'

The combine_matching_pkls function will combine all prediction and ground truth .pkl files into a single .pkl file. This is the .pkl file that is used for submission to the leaderboard. Running evaluate_pkls will the predicted tracks across four metrics: HOTA-Temporal, HOTA, timestamp-level F1, and scenario-level F1.

In [ ]:
from refAV.eval import combine_matching_pkls, evaluate_pkls

eval_output_dir = Path(f'output/evaluation/val')
combine_matching_pkls(SM_DATA_DIR / 'val', SM_PRED_DIR / 'val', eval_output_dir)
evaluate_pkls(eval_output_dir / 'combined_predictions.pkl', eval_output_dir / 'combined_gt.pkl')


If the evaluate function executes successfully, your predictions .pkl file is ready to submit to the EvalAI server for evaluation. Create a profile at EvalAI [EvalAI](https://eval.ai/) in to receive an account token. This code will submit to the validation set.

```bash
pip install evalai
evalai set_token <EvalAI_account_token>
evalai challenge 2469 phase 4899 submit --file output/evaluation/val/combined_predictions.pkl --large
```